In [32]:
import gym
import numpy as np
import tensorflow as tf 
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [5]:
env = gym.make('CartPole-v0')

# Basic policy

In [6]:
def basic_policy(obs):
    '''
    take the observation which represent the status as input 
    and return the action which will be taken  
    '''
    angle=obs[2]
    return 0 if angle<0 else 1

In [7]:
totals=[] # save a sequence of action 

In [8]:
for episode in range(500):
    episode_rewards=0
    obs=env.reset()
    for step in range(1000):
        action = basic_policy(obs)
        obs,reward,done,info = env.step(action)
        episode_rewards+=reward
        if done:
            break
    totals.append(episode_rewards)

In [9]:
a = np.array(totals)

In [10]:
a.mean()

41.694

In [11]:
a.max()

65.0

# Neural Network Policy
using the neural network to learn what is the next action to be taken. 

In [12]:
tf.reset_default_graph()

In [13]:
n_inputs = 4
n_hidden = 4
n_outputs = 1
initializer = tf.contrib.layers.variance_scaling_initializer()

In [14]:
x = tf.placeholder(tf.float32,shape=[None,n_inputs])
hidden = fully_connected(x,n_hidden,activation_fn=tf.nn.elu,weights_initializer=initializer)
logits = fully_connected(hidden,n_outputs,activation_fn=None,weights_initializer=initializer)
outputs = tf.nn.sigmoid(logits)

In [15]:
p_left_and_right = tf.concat(axis=1,values=[outputs,1-outputs])# if the shape of outputs is n*1 which n is the number of samples, values will be 2*n*1,p_left_and_right will be n*2
action = tf.multinomial(tf.log(p_left_and_right),num_samples=1)# return a random action draw from the different class with the probability in p_left_and_right

In [16]:
'''
# to under stand muitinomial
samples = tf.multinomial(tf.log([[70,30.,1],[1,1,1]]), 2)
with tf.Session() as sess:
    print(sess.run(samples)) 
'''

'\n# to under stand muitinomial\nsamples = tf.multinomial(tf.log([[70,30.,1],[1,1,1]]), 2)\nwith tf.Session() as sess:\n    print(sess.run(samples)) \n'

In [17]:
y = 1.0-tf.to_float(action)
learning_rate = 0.01
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]

In [18]:
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32,shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder,variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

In [19]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [20]:
def discount_rewards(rewards,discount_rate):
    discount_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step]+cumulative_rewards*discount_rate
        discount_rewards[step] = cumulative_rewards
    return discount_rewards

In [21]:
def discount_and_normalize_rewards(all_rewards,discount_rate):
    all_discounted_rewards = [discount_rewards(rewards,discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards-reward_mean)/reward_std for discounted_rewards  in all_discounted_rewards]

In [22]:
'''
def discount_reward(reward,discount_rate):
    discount_reward = reward
    total=0
    for i in range(len(reward)):
        total = reward[len(reward)-i-1]+total*discount_rate
        discount_reward[len(reward)-i-1] = total
    return discount_reward
'''

'\ndef discount_reward(reward,discount_rate):\n    discount_reward = reward\n    total=0\n    for i in range(len(reward)):\n        total = reward[len(reward)-i-1]+total*discount_rate\n        discount_reward[len(reward)-i-1] = total\n    return discount_reward\n'

In [50]:
n_itertions = 500
n_max_step = 100
n_game_per_update = 10
save_iterations = 10
discount_rate = 0.95

In [51]:
with tf.Session() as sess:
    init.run()
    for iteration in range(n_itertions):
        all_rewards = []
        all_gradients = []
        for game in range(n_game_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            for step in range(n_max_step):
                action_val,gradients_val = sess.run([action,gradients],feed_dict={x:obs.reshape(1,n_inputs)})
                obs,reward,done,info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
        all_rewards.append(current_rewards)
        all_gradients.append(current_gradients)
    all_rewards = discount_and_normalize_rewards(all_rewards,discount_rate)
    feed_dict={}
    for var_index,grad_placeholder in enumerate(gradient_placeholders ):
        mean_gradients = np.mean([reward*all_gradients[game_index][step][var_index] 
                                  for game_index,rewards in enumerate(all_rewards)],axis=0)
        feed_dict[grad_placeholder] = mean_gradients
    sess.run(training_op, feed_dict=feed_dict)
    if iteration%save_iterations==0:
        saver.save(sess,'./nn_policy.ckpt')

13
19
37
17
26
37
18
29
23
18
22
52
27
15
20
28
39
44
13
41
12
40
31
21
14
16
25
29
37
13
24
35
25
34
29
38
25
11
9
27
28
25
15
24
15
50
36
27
35
28
